<a href="https://colab.research.google.com/github/AZZIELM/movielens.org_recomendacion/blob/main/Recomendaci%C3%B3n_pel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Conocer los datasets

In [1]:
import pandas as pd
import numpy as np


In [2]:
peliculas= pd.read_csv('https://raw.githubusercontent.com/AZZIELM/movielens.org_recomendacion/main/movies.csv')
peliculas.columns = ['peliculaId','titulo','generos']
peliculas = peliculas.set_index('peliculaId')
peliculas.head()

,titulo,generos
peliculaId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [3]:
peliculas.describe()

,titulo,generos
count,9742,9742
unique,9737,951
top,Emma (1996),Drama
freq,2,1053


In [4]:
notas = pd.read_csv('https://raw.githubusercontent.com/AZZIELM/movielens.org_recomendacion/main/ratings.csv')
notas.columns=['usuarioId','peliculaId','nota','momento']
notas.head()

,usuarioId,peliculaId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
notas.describe()

,usuarioId,peliculaId,nota,momento
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


#Primer intento de recomendación

In [6]:
notas.value_counts("peliculaId")

peliculaId
356       329
318       317
296       307
593       279
2571      278
         ... 
4093        1
4089        1
58351       1
4083        1
193609      1
Name: count, Length: 9724, dtype: int64

In [7]:
peliculas.loc[356]

titulo          Forrest Gump (1994)
generos    Comedy|Drama|Romance|War
Name: 356, dtype: object

In [8]:
total_de_votos= notas.value_counts("peliculaId")

In [9]:
peliculas['total_de_votos']= total_de_votos
peliculas.sort_values('total_de_votos', ascending=False).head(10)

,titulo,generos,total_de_votos
peliculaId,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0
110,Braveheart (1995),Action|Drama|War,237.0
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0


#Segundo intento - Considerando las notas

In [10]:
media_votos = notas.groupby('peliculaId').mean()['nota']
peliculas['media_votos'] = media_votos
peliculas.head()

,titulo,generos,total_de_votos,media_votos
peliculaId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818
3,Grumpier Old Men (1995),Comedy|Romance,52.0,3.259615
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0,2.357143
5,Father of the Bride Part II (1995),Comedy,49.0,3.071429


In [11]:
peliculas_con_mas_50_votos = peliculas.query('total_de_votos >= 50')
peliculas_con_mas_50_votos.sort_values('media_votos', ascending=False).head(10)

,titulo,generos,total_de_votos,media_votos
peliculaId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041
904,Rear Window (1954),Mystery|Thriller,84.0,4.261905
1221,"Godfather: Part II, The (1974)",Crime|Drama,129.0,4.259690
48516,"Departed, The (2006)",Crime|Drama|Thriller,107.0,4.252336
1213,Goodfellas (1990),Crime|Drama,126.0,4.250000


#Tercer intento -  Considerando las peliculas que el usuario miró

In [12]:
peliculas_que_mire = [1,19,21,10,2]
peliculas.loc[peliculas_que_mire]

,titulo,generos,total_de_votos,media_votos
peliculaId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
19,Ace Ventura: When Nature Calls (1995),Comedy,88.0,2.727273
21,Get Shorty (1995),Comedy|Crime|Thriller,89.0,3.494382
10,GoldenEye (1995),Action|Adventure|Thriller,132.0,3.496212
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818


In [13]:
aventura_infantil_fantasia=peliculas_con_mas_50_votos.query("generos == 'Adventure|Children|Fantasy'")
aventura_infantil_fantasia.drop(peliculas_que_mire, errors= 'ignore').sort_values('media_votos',ascending=False)

,titulo,generos,total_de_votos,media_votos
peliculaId,,,,
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,107.0,3.761682
41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,62.0,3.443548


In [14]:
# Colaborative filtering vs Content - based filtering

#Generalizando el cálculo de distancias

In [15]:
notas.head()

,usuarioId,peliculaId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [16]:
usuario1 = notas.query("usuarioId == 1")[['peliculaId','nota']].set_index('peliculaId')
usuario4 = notas.query("usuarioId == 4")[['peliculaId','nota']].set_index('peliculaId')
diferencias = usuario1.join(usuario4, lsuffix='_izq', rsuffix='_der').dropna()
diferencias['nota_izq']

peliculaId
47      5.0
235     4.0
260     5.0
296     3.0
441     4.0
457     5.0
553     5.0
593     4.0
608     5.0
648     3.0
919     5.0
1025    5.0
1060    4.0
1073    5.0
1080    5.0
1136    5.0
1196    5.0
1197    5.0
1198    5.0
1213    5.0
1219    2.0
1265    4.0
1282    5.0
1291    5.0
1500    4.0
1517    5.0
1580    3.0
1617    5.0
1732    5.0
1967    4.0
2078    5.0
2174    4.0
2395    5.0
2406    4.0
2571    5.0
2628    4.0
2692    5.0
2858    5.0
2959    5.0
2997    4.0
3033    5.0
3176    1.0
3386    5.0
3489    4.0
3809    4.0
Name: nota_izq, dtype: float64

In [17]:
def notas_del_usuario(usuarioId):
  notas_del_usuario= notas.query("usuarioId == %d" % usuarioId)[['peliculaId','nota']]
  notas_del_usuario= notas_del_usuario.set_index('peliculaId')
  return notas_del_usuario

In [18]:
def distancia_de_vectores(a,b):
  return np.linalg.norm(a-b)

In [19]:
def distancia_entre_usuarios(usuario_id1,usuario_id2):
  notas1 = notas_del_usuario(usuario_id1)
  notas2 = notas_del_usuario(usuario_id2)
  diferencias = notas1.join(notas2, lsuffix='_izq', rsuffix='_der').dropna()
  return distancia_de_vectores(diferencias['nota_izq'], diferencias['nota_der'])

In [20]:
distancia_entre_usuarios(1,4)

11.135528725660043

#Distancia de un usuario de referencia con el resto de usuarios

In [21]:
len(notas['usuarioId'].unique())

610

In [22]:
def distancia_entre_usuarios(usuario_id1,usuario_id2):
  notas1 = notas_del_usuario(usuario_id1)
  notas2 = notas_del_usuario(usuario_id2)
  diferencias = notas1.join(notas2, lsuffix='_izq', rsuffix='_der').dropna()
  diferencia = distancia_de_vectores(diferencias['nota_izq'], diferencias['nota_der'])
  return [usuario_id1,usuario_id2,diferencia]

In [23]:
def distancia_de_todos(usuario_ref):
  usuarios = notas['usuarioId'].unique()
  distancias = []
  for usuario in usuarios:
    informaciones = distancia_entre_usuarios(usuario_ref,usuario)
    distancias.append(informaciones)
  distancias = pd.DataFrame(distancias, columns=['usuarios_ref','usuarios','distancia'])
  return distancias

In [24]:
distancia_de_todos(1).head()

,usuarios_ref,usuarios,distancia
0,1,1,0.000000
1,1,2,1.414214
2,1,3,8.200610
3,1,4,11.135529
4,1,5,3.741657


In [25]:
distancia_de_todos(1).sort_values('distancia').head(20)

,usuarios_ref,usuarios,distancia
0,1,1,0.000000
577,1,578,0.000000
76,1,77,0.000000
84,1,85,0.000000
174,1,175,0.000000
555,1,556,0.000000
252,1,253,0.000000
290,1,291,0.000000
305,1,306,0.000000
495,1,496,0.000000


In [26]:
notas_del_usuario(1).join(notas_del_usuario(77), lsuffix='_izq', rsuffix='_der').dropna()

,nota_izq,nota_der
peliculaId,,
260,5.0,5.0
1196,5.0,5.0
1198,5.0,5.0
1210,5.0,5.0
2571,5.0,5.0
3578,5.0,5.0


#Refinando nuestra función de distancia para no considerar usuarios distantes

In [27]:
def distancia_entre_usuarios(usuario_id1,usuario_id2,minimo=5):
  notas1 = notas_del_usuario(usuario_id1)
  notas2 = notas_del_usuario(usuario_id2)
  diferencias = notas1.join(notas2, lsuffix='_izq', rsuffix='_der').dropna()
  if len(diferencias)< minimo:
    return [usuario_id1,usuario_id2,10000]
  diferencia = distancia_de_vectores(diferencias['nota_izq'], diferencias['nota_der'])
  return [usuario_id1,usuario_id2,diferencia]

In [28]:
def distancia_de_todos(usuario_ref):
  usuarios = notas['usuarioId'].unique()
  distancias = []
  for usuario in usuarios:
    informaciones = distancia_entre_usuarios(usuario_ref,usuario)
    distancias.append(informaciones)
  distancias = pd.DataFrame(distancias, columns=['usuarios_ref','usuarios','distancia'])
  return distancias

In [29]:
distancia_de_todos(1).sort_values('distancia').head(20)

,usuarios_ref,usuarios,distancia
0,1,1,0.000000
76,1,77,0.000000
510,1,511,0.500000
365,1,366,0.707107
522,1,523,1.000000
48,1,49,1.000000
8,1,9,1.000000
257,1,258,1.000000
318,1,319,1.118034
397,1,398,1.224745


#Identificando el usuario más próximo

In [30]:
def mas_proximos_usuarios_ref(usuario_ref):
  mas_proximos = distancia_de_todos(usuario_ref)
  mas_proximos = mas_proximos.sort_values('distancia')
  mas_proximos = mas_proximos.set_index('usuarios')
  mas_proximos = mas_proximos.drop(usuario_ref)
  return mas_proximos

mas_proximos_usuarios_ref(1)

,usuarios_ref,distancia
usuarios,,
77,1,0.000000
511,1,0.500000
366,1,0.707107
523,1,1.000000
49,1,1.000000
...,...,...
190,1,10000.000000
60,1,10000.000000
576,1,10000.000000


In [31]:
def distancia_entre_usuarios(usuario_id1,usuario_id2,minimo=5):
  notas1 = notas_del_usuario(usuario_id1)
  notas2 = notas_del_usuario(usuario_id2)
  diferencias = notas1.join(notas2, lsuffix='_izq', rsuffix='_der').dropna()
  if len(diferencias)< minimo:
    return None
  diferencia = distancia_de_vectores(diferencias['nota_izq'], diferencias['nota_der'])
  return [usuario_id1,usuario_id2,diferencia]

In [32]:
def distancia_de_todos(usuario_ref, numero_usuario_analizar= None):
  usuarios = notas['usuarioId'].unique()
  distancias = []
  if numero_usuario_analizar:
    usuarios = usuarios[:numero_usuario_analizar]
  for usuario in usuarios:
    informaciones = distancia_entre_usuarios(usuario_ref,usuario)
    distancias.append(informaciones)
  distancias = list(filter(None,distancias))
  distancias = pd.DataFrame(distancias, columns=['usuarios_ref','usuarios','distancia'])
  return distancias

In [33]:
def mas_proximos_usuarios_ref(usuario_ref,numero_usuario_analizar= None):
  mas_proximos = distancia_de_todos(usuario_ref,numero_usuario_analizar)
  mas_proximos = mas_proximos.sort_values('distancia')
  mas_proximos = mas_proximos.set_index('usuarios')
  mas_proximos = mas_proximos.drop(usuario_ref)
  return mas_proximos

In [34]:
similares = mas_proximos_usuarios_ref(1,100)
similar = similares.iloc[0].name
similar

77

#Recomendando las peliculas del más próximo

In [35]:
def sugerencias(usuario_ref,numero_usuario_analizar = None):
  notas_usuario_ref = notas_del_usuario(usuario_ref)
  similares = mas_proximos_usuarios_ref(usuario_ref,numero_usuario_analizar)
  similar = similares.iloc[0].name

  notas_del_similar = notas_del_usuario(similar)
  notas_del_similar = notas_del_similar.drop(notas_usuario_ref.index, errors = 'ignore')
  recomendaciones = notas_del_similar.sort_values('nota', ascending=False)
  return recomendaciones.join(peliculas)

In [36]:
sugerencias(1,100).head(10)

,nota,titulo,generos,total_de_votos,media_votos
peliculaId,,,,,
8636,5.0,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX,79.0,3.803797
58559,5.0,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,149.0,4.238255
33794,5.0,Batman Begins (2005),Action|Crime|IMAX,116.0,3.862069
4993,5.0,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,198.0,4.106061
5349,5.0,Spider-Man (2002),Action|Adventure|Sci-Fi|Thriller,122.0,3.540984
5378,5.0,Star Wars: Episode II - Attack of the Clones (...,Action|Adventure|Sci-Fi|IMAX,92.0,3.157609
8961,5.0,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,125.0,3.836000
5816,5.0,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy,102.0,3.598039
5952,5.0,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,188.0,4.021277


#Sugerencias en función de varios usuarios

In [48]:
def knn(usuario_ref,k_mas_proximos = 10,numero_usuario_analizar= None):
  mas_proximos = distancia_de_todos(usuario_ref,numero_usuario_analizar)
  mas_proximos = mas_proximos.sort_values('distancia')
  mas_proximos = mas_proximos.set_index('usuarios')
  mas_proximos = mas_proximos.drop(usuario_ref, errors='ignore')
  return mas_proximos.head(k_mas_proximos)

In [49]:
  def sugerencias(usuario_ref,k_mas_proximos = 10,numero_usuario_analizar = None):
    notas_usuario_ref = notas_del_usuario(usuario_ref)
    similares = knn(usuario_ref,k_mas_proximos,numero_usuario_analizar)
    usuarios_similares = similares.index
    notas_similares = notas.set_index("usuarioId").loc[usuarios_similares]
    notas_similares = notas_similares.groupby("peliculaId").mean()[['nota']]
    recomendaciones = notas_similares.drop(notas_usuario_ref.index, errors = 'ignore')
    recomendaciones = recomendaciones.sort_values('nota', ascending=False)
    return recomendaciones.join(peliculas)


In [50]:
sugerencias(2,10,50).head()

,nota,titulo,generos,total_de_votos,media_votos
peliculaId,,,,,
597,5.0,Pretty Woman (1990),Comedy|Romance,135.0,3.485185
51255,5.0,Hot Fuzz (2007),Action|Comedy|Crime|Mystery,61.0,4.000000
1500,5.0,Grosse Pointe Blank (1997),Comedy|Crime|Romance,59.0,3.610169
1304,5.0,Butch Cassidy and the Sundance Kid (1969),Action|Western,62.0,3.943548
1954,5.0,Rocky (1976),Drama,64.0,3.828125


##Probando un nuevo usuario

In [39]:
def nuevo_usuario(datos):
  usuario_id = notas['usuarioId'].max()+1
  notas_usuario_nuevo= pd.DataFrame(datos, columns=['peliculaId','nota'])
  notas_usuario_nuevo['usuarioId'] = usuario_id
  return pd.concat([notas,notas_usuario_nuevo])

In [40]:
notas = nuevo_usuario([[122904,2],[1246,5],[2529,2]])
notas.tail()

,usuarioId,peliculaId,nota,momento
100834,610,168252,5.0,1.493846e+09
100835,610,170875,3.0,1.493846e+09
0,611,122904,2.0,NaN
1,611,1246,5.0,NaN
2,611,2529,2.0,NaN


#Filtrando películas con más de 50 votos

In [41]:
notas = notas.set_index('peliculaId').loc[peliculas_con_mas_50_votos.index]

In [42]:
len(notas)

41363

In [43]:
notas = notas.reset_index()
notas.head()

,peliculaId,usuarioId,nota,momento
0,1,1,4.0,9.649827e+08
1,1,5,4.0,8.474350e+08
2,1,7,4.5,1.106636e+09
3,1,15,2.5,1.510578e+09
4,1,17,4.5,1.305696e+09


In [51]:
sugerencias(610)

,nota,titulo,generos,total_de_votos,media_votos
peliculaId,,,,,
1391,5.000000,Mars Attacks! (1996),Action|Comedy|Sci-Fi,86.0,3.093023
17,5.000000,Sense and Sensibility (1995),Drama|Romance,67.0,3.776119
25,5.000000,Leaving Las Vegas (1995),Drama|Romance,76.0,3.625000
36,5.000000,Dead Man Walking (1995),Crime|Drama,67.0,3.835821
1203,5.000000,12 Angry Men (1957),Drama,57.0,4.149123
2329,4.500000,American History X (1998),Crime|Drama,129.0,4.217054
2657,4.500000,"Rocky Horror Picture Show, The (1975)",Comedy|Horror|Musical|Sci-Fi,63.0,3.341270
2174,4.500000,Beetlejuice (1988),Comedy|Fantasy,91.0,3.472527
2502,4.500000,Office Space (1999),Comedy|Crime,94.0,4.090426


#Considerando un valor mínimo de usuarios knn para las recomendaciones

In [58]:
def knn(usuario_ref,k_mas_proximos=10,numero_usuario_analizar= None):
  mas_proximos = distancia_de_todos(usuario_ref,numero_usuario_analizar)
  mas_proximos = mas_proximos.sort_values('distancia')
  mas_proximos = mas_proximos.set_index('usuarios')
  mas_proximos = mas_proximos.drop(usuario_ref, errors='ignore')
  return mas_proximos.head(k_mas_proximos)


In [59]:
  def sugerencias(usuario_ref,k_mas_proximos = 10,numero_usuario_analizar= None):
    notas_usuario_ref = notas_del_usuario(usuario_ref)
    similares = knn(usuario_ref,k_mas_proximos,numero_usuario_analizar)
    usuarios_similares = similares.index
    notas_similares = notas.set_index("usuarioId").loc[usuarios_similares]
    cant_veces = notas_similares.groupby('peliculaId').count()[['nota']]
    filtro_minimo= k_mas_proximos/2
    notas_similares = notas_similares.groupby("peliculaId").mean()[['nota']]
    recomendaciones = notas_similares.drop(notas_usuario_ref.index, errors = 'ignore')
    recomendaciones = recomendaciones.join(cant_veces, lsuffix = '_media', rsuffix = '_cant_veces')
    recomendaciones = recomendaciones.query('nota_cant_veces >= %.2f' % filtro_minimo)
    recomendaciones = recomendaciones.sort_values('nota_media', ascending=False)
    return recomendaciones.join(peliculas)


In [61]:
sugerencias(610,5,50).head(10)

,nota_media,nota_cant_veces,titulo,generos,total_de_votos,media_votos
peliculaId,,,,,,
292,4.666667,3,Outbreak (1995),Action|Drama|Sci-Fi|Thriller,101.0,3.425743
590,4.333333,3,Dances with Wolves (1990),Adventure|Drama|Western,164.0,3.835366
253,4.000000,3,Interview with the Vampire: The Vampire Chroni...,Drama|Horror,109.0,3.458716
141,3.666667,3,"Birdcage, The (1996)",Comedy,86.0,3.494186
161,3.666667,3,Crimson Tide (1995),Drama|Thriller|War,103.0,3.635922
165,3.666667,3,Die Hard: With a Vengeance (1995),Action|Crime|Thriller,144.0,3.555556
185,3.666667,3,"Net, The (1995)",Action|Crime|Thriller,112.0,3.040179
316,3.333333,3,Stargate (1994),Action|Adventure|Sci-Fi,140.0,3.375000
434,3.333333,3,Cliffhanger (1993),Action|Adventure|Thriller,101.0,3.034653
